# Modele nieliniowe

<h1 id=tocheading>Spis treści</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
sns.set(font_scale=2.5)

In [3]:
# -*- coding: utf-8 -*-
"""
Demo of unicode support in text and labels.
"""
from __future__ import unicode_literals
from __future__ import print_function
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.datasets import load_svmlight_file, load_boston, load_breast_cancer
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, matthews_corrcoef, confusion_matrix
from sklearn import preprocessing
from sklearn.datasets import load_boston, load_diabetes, load_linnerud, make_regression
from sklearn.datasets import make_friedman1, make_friedman2, make_friedman3, make_sparse_uncorrelated
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [4]:
from astroML.datasets import fetch_rrlyrae_combined
from astroML.utils import split_samples, completeness_contamination

# Ograniczenia modeli
* modele liniowe mają ograniczenia z powodu klątwy wymiarowości
* niewystarczające są __ustalone__ funkcje bazowe
  * zły wybór może powodować overfitting  
  * trzeba __adaptować__ funkcje do konkretnych danych
  * ograniczyć liczbę funkcji bazowych

* alternatywne podejście
  * ustalić funkcje bazowe z góry - niewielką liczbę, zwykle __jedną__
  * niech funkcje __adaptują__ się w trakcie nauki
  * model jest zwykle znacznie mniejszy
  * funkcja ___likelihood_ nie będzie już wypukła__
    * konieczność przeglądnięcia większej liczby modeli

## Neuron biologiczny

  <img src="../mum_figures/neuron.png" width="100%"/>
  
  <img src="../mum_figures/synapse.jpg" width="100%"/>

![Potential](file:///Users/igorpodolak/Dropbox/mum_figures/potential.gif)

![Awesome cat gif](http://media.giphy.com/media/7z2oyDXIMEs8w/giphy.gif)

![Potential](http://github.com/gmum/ml2017/wyklady/mum_figures/potential.gif)

<img src="../mum_figures/potential.jpg" width="100%"/>

## Sieci z propagacją sygnału (feed-forward)
<img src="../mum_figures/mlp.png" width="100%"/>
* funkcje bazowe modelu
  * muszą __zależeć__ od parametrów
  * parametry powinny się zmieniać w trakcie uczenia
  * współczynniki $w_j$ także są modyfikowalne
* $M$ liniowych kombinacji wejść $$v_j=\sum_i^Dw_{ji}x_i+w_{j0}$$
gdzie $M$ jest liczbą __"neuronów"__ w pierwszej __warstwie__ modelu
  * $w_{j0}$ to __bias__ $j$-tego neuronu
  * $v_j$ to __aktywacje__
  * teraz użyta jest __funkcja aktywacji__ $\varphi()$ (różniczkowalna) $$z_j=\varphi(v_j)$$
    * funkcje aktywacji są zwykle funkcjami sigmoidalnymi, np.
    $$\frac{1}{1+\exp({-v})}, \tanh(-v)$$
  * neurony w warstwach, które nie są ani wejściową ani ukrytą nazywamy __ukrytymi__
  
  
* obliczenia powtarzają się w kolejnych warstwach
* końcowa wartość jest jako, dla sieci z jedną warstwą ukrytą
$$y_k(x)=\sigma\left(\sum_k^Mw_kj\sigma\left(\sum_i^Dw_{ji}x_i+w_{j0}\right)+w_{k0}\right)$$
  * jeśli problem regresji, to końcowa funkcja aktywacji jest identycznością
  * sieć neuronowa jest __nieliniową__ funkcją wejść $x_i$
    * wartość bias włączamy do wektora $w$ __rozszerzając__ wejście o $x_0=1$

* sygnał jest przesyłany do przodu, stąd nazwa
* sieć wielowarstwowa nazywana __wielowarstwowym perceptronem__
  * zaproponowany przez McCullocha i Pittsa __perceptron__ 
    * wykorzystywał __progową__ funkcję aktywacji (f. Heaviside'a)
    * nie zawierał warstw ukrytych

* sieć z __liniowymi__ f. aktywacji w w. ukrytych jest redukowalna do sieci __bez__ warstw ukrytych
* sieci warstwowe można uogólnić przez dodanie __połączeń skrótowych__

* nazewnictwo
  * istnieje wiele zwyczajów: sieć na rysunku może być nazywana 
    * 4-warstwową (bo liczba wszystkich warstw neuronów), 
    * 3-warstwową (bo liczba warstw adaptowalnych parametrów), 
    * 2-warstwową (bo liczba warstw ukrytych)
  * Bishop preferuje drugie podejście
  * jasne będzie "sieć z dwoma warstwami ukrytymi"

## Sieci neuronowe jako uniwersalny aproksymator
> Sieć neuronowa z jedną warstwą ukrytą neuronów z nieliniowymi ciągłymi monotonicznie rosnącymi funkcjami aktywacji może dowolnie dokładnie dla $\epsilon>0$ aproksymować dowolną funkcję ciągła zdefiniowaną na hiperkostce $[0, 1]^m$.

> Twierdzenie jest o _istnieniu_ takiej sieci, a nie mówi jak znaleźć odpowiednią liczbę neuronów.

## Funkcja kosztu: regresja
* niech przewidywana pojedyncza zmienna $t$ ma rozkład normalny zależny od wejściowego atrybutu $x$ $$p(t\mid x)=\mathcal{N}(t\mid y(x, w), \sigma)$$
gdzie
  * $y(x, w)$ jest wyjściem sieci
  * $\sigma$ jest wariancją
* niech $X=\{x_1,x_2,\dots,x_N\}$ będzie zbiorem $N$ __niezależnych__ przykładów
* dla zbioru wartości spodziewanych (target) $\{t_1,\dots,t_N\}$ można utworzyć funkcję __likelihood__ 
$$p(t\mid X,w,\sigma)=\prod_{n=1}^Np(t_n\mid x_n,w,\sigma)$$

* biorąc __ujemny logarytm__ dostajemy __log-likelihood__
$$\frac{1}{2\sigma}\sum_{n=1}^N\left(y(x_n,w)-t_n\right)^2+\frac{N}{2}\ln\sigma +\frac{N}{2}\ln(2\pi)$$
* maksymalizacja likelihood jest równoważna __minimalizacji__ log-likelihood, co jest identyczne z minimalizacją funkcji kosztu
$$E(w)=\frac{1}{2}\sum_{n=1}^N\left(y(x_n,w)-t_n\right)^2$$
  * nieliniowość funkcji $y(x,w)$ powoduje, że $E(w)$ __nie jest wypukła__
  * funkcja kosztu będzie miała __minima lokalne__
  * po znalezieniu wektora $w^\ast$ można znaleźć $\sigma$ jako
  $$\sigma=\frac{1}{N}\sum_{n=1}^N\left(y(x_n,w^\ast)-t_n\right)^2$$



## Funkcja kosztu: klasyfikacja
* przypadek klasyfikacji binarnej:
  * pojedynczy neuron wyjsciowy z $\sigma()$ __logistyczną__
  $$y=\sigma(v)=\frac{1}{1+\exp(-v)}$$
  * niech $t=1$ jeśli $x\in C_1$; $t=2$ jeśli $x\in C_2$
  * niech $y(x,w)$ będzie __prawdopodobieństwem__, że $x\in C_1$
  * $1-y(x,w)$ prawdopodobieństwem, że $x\in C_2$
  
* rozkład warunkowy wyjść jest rozkładem Bernoulliego
$$p(t\mid x)=y(x,w)^t\left(1-y(x,w)\right)^{1-t}$$

* dla zbioru wszystkich przykładów mamy likelihood
$$p(t\mid X,w)=\prod_ny(x_n,w)^t_n\left(1-y(x_n,w)\right)^{1-t_n}$$
* znowu biorąc ujemny logarytm dostajemy log-likelihood i funkcję kosztu
$$E(w)=-\sum_{n=1}^N\left[t_n\ln\,y_n+(1-t_n)\ln\,(1-y_n)\right]$$
nazywaną __entropią krzyżową__
  
  * dla klasyfikacji użycie entropii krzyżowej daje lepsze wyniki niż użycie funkcji kwadratowej kosztu
* dla problemu wieloklasowego $t\in\{C_1,\dots,C_K\}$ mamy
$$E(w)=-\sum_{n=1}^N\sum_{k=1}^K\left[t_{nk}\ln\,y_k(x_n,w)+(1-t_{nk})\ln\,(1-y_k(x_n,w))\right]$$
* dla problemów klasyfikacji wieloklasowej mamy
  $$y_k(x,w)=p(t_k=1\mid x)$$
  * to skraca wyrażenie entropii krzyżowej do
  $$E(w)=-\sum_{n=1}^N\sum_{k=1}^Kt_n\ln\,y_k(x_n,w)$$
  * dla problemu wieloklasowego aktywacją wyjściową jest funkcja softmax
  $$y_k(x,w)=\frac{\exp(v_k(x,w))}{\sum_{j=1}^K\exp(v_j(x,w))}$$

### Cechy
* podstawowa różnica do modeli liniowych: sieć neuronowa __mapuje przestrzeń wejściową do przestrzeni ukrytej gdzie znajduje rozwiązanie__
  * __nieliniowe__ rzutowanie do przestrzeni ukrytej zwiększa prawdopodobieństwa liniowej separowalności

* aktywacje warstw ukrytych dla danego wejścia nazywamy cechami
  * w modelach każda przynależność do klasy jest rozwiązywana jako __całkowicie osobny__ problem
  * w sieciach podproblemy przynależności do klas __dzielą cechy__

## Uczenie sieci warstwowych neuronowych
* zadaniem jest minimalizacja $E(w)$
* drobna zmiana $w$ powoduje __presunięcie__ się po powierzchni funkcji kosztu
  * dualność: każde rozwiązanie odpowiada punktowi w przestrzeni wag
* przestrzeń rozwiązań jest zwykle bardzo wysoko wymiarowa
  * istnieje wiele punktów o $\nabla{}E(w)\simeq0$
  * to minima/maksima lokalne/globalne, punkty siodłowe

### Lokalna aproksymacja kwadratowa
* rozwinięcie funkcji kosztu w szereg Taylora
$$E(w)\simeq E(w_0)+(w-w_0)g+\frac{1}{2}(w-w_0)H(w-w_0)$$
gdzie 
$$\begin{align}
g=&\frac{\partial E}{\partial w}\\
H=&\frac{\partial^2 E}{\partial w_i\,\partial w_j}
\end{align}$$

* stąd
$$\nabla E\simeq g+H(w-w_0)$$

## Uczenie gradientowe
* uczenie małymi krokami
$$w(t+1)=w(t)-\eta\nabla E(w(t))$$
gdzie $\eta$ jest __współczynnikiem nauczania__

* to metoda __epoka po epoce__ (ang. batch)
  * każdy krok poprawy wymaga wielu obliczeń
  * każda propagacja wprzód używa starych wag
  * wiele przykładów dzieli wspólne cechy, stąd utrata wielu możliwych informacji
  * uczenie po epokach może powodować krążenie w przestrzeni rozwiązań
  * usprawnienie: metoda gradientów sprzężonych zapewniająca poprawy
    * każda poprawa jest ortogonalna do poprzednich
    * teoretycznie roziązanie po co najwyżej tylu krokach ile jest wag

* epoka __przykład po przykładzie__ (ang. stochastic gradient descent)
$$E(w)=\sum_nE_n(w)$$
  * poprawa po każdym przykładzie
  $$w(t+1)=w(t)-\eta\nabla E_n(w(t))$$
  * uwzględnia wiedzę z innych przykładów od razu
  * pozwala na łatwiejsze omijania minimów lokalnych
 
* __mini batch__
  * uczenie małymi grupami przykładów
  * najbardziej efektywne

## Wsteczna propagacja błędów
* sieci przechodziły przez wzloty i upadki
* zapowiadano (Minsky i Papert w _Perceptron_) śmierć ze względu na __niemożność__ uczenia sieci z warstwami ukrytymi
* problem z liczeniem gradientu dla warstw ukrytych
* algorytm wstecznej propagacji zaproponowany przez Werbosa


* w trakcie uczenia obliczana jest duża liczba pochodnych funkcji kosztu __względem__ wag
* wsteczna propagacja jest __efektywnym__ machenizmem ich obliczania
  * pochodne (błędy) są przesyłane wstecz w sieci
* drugim elementem jest obliczenie i modyfikacja wartości wag

* schemat może być uzyty do szerokiego wachlarza architektur sieci

### Pochodne


## Regularyzacja